In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms, datasets
from scipy.stats import norm
import preprocessingWithMissingvalues
import pandas as pd

import calendar
import copy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import calendar
import datetime
from datetime import datetime
from sklearn.model_selection import train_test_split

%matplotlib inline

In [3]:
#read the data
car_df= pd.read_csv('./cardata.csv')


In [4]:
featureName= list(car_df.columns.values) #get the column/feature name of the data
#handle the ambiguity in the data
dayIndex= car_df.shape[0] + 2   #set the value other than 0-15419
monthIndex= car_df.shape[0] + 1 #set the value other than 0-15419

for i in range(car_df.shape[0]):
    if(car_df['DayOfWeekClaimed'][i]=='0'):
        dayIndex= i
        print("index: ", i)
    if(car_df['MonthClaimed'][i]=='0'):
        monthIndex= i
        print("\nindex:",i)

#create dataframe for keeping new form of dataset
carData= pd.DataFrame(columns=featureName)
j= 0
#put all the data into new dataframe
for i in range(car_df.shape[0]):
    if(i!=dayIndex):
        carData.loc[j]= car_df.loc[i]
        j += 1

#replace map
replace_Month = {'Month':{'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
                          'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}}

replace_MonthClaimed = {'MonthClaimed':{'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
                          'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}}

carData.replace(replace_Month,inplace=True)
carData.replace(replace_MonthClaimed,inplace=True)

#function to calculate the no of days passed between the accident and the claims.
# Reporting Gap:
def get_day(year,month,weekOfMonth,dayOfWeek):
    count = 0
    c = calendar.TextCalendar(calendar.SUNDAY)
    l = []
    year= int(year)
    month=int(month)
    for i in c.itermonthdates(year,month):
        l.append(i)
    for j in range(len(l)):
        day = calendar.day_name[l[j].weekday()]
        if day == dayOfWeek:
            count += 1
        if count == weekOfMonth:
            return l[j]
            break

def differ_days(date1,date2):
    a = date1
    b = date2
    return (a - b).days



index:  1516

index: 1516


In [5]:
day_diff = np.zeros((carData.shape[0],1))
i = 0
for i in range(carData.shape[0]):

    if(carData['MonthClaimed'][i]-carData['Month'][i]) < 0:
        year2 = carData['Year'][i] + 1
        month2 = carData['MonthClaimed'][i]
        week2 = carData['WeekOfMonthClaimed'][i]
        day2 = carData['DayOfWeekClaimed'][i]
        year1 = carData['Year'][i]
        month1 = carData['Month'][i]
        week1 = carData['WeekOfMonth'][i]
        day1 = carData['DayOfWeek'][i]
        day_diff[i] = differ_days(get_day(year2,month2,week2,day2),get_day(year1,month1,week1,day1))
    else:
        year2 = carData['Year'][i]
        month2 = carData['MonthClaimed'][i]
        week2 = carData['WeekOfMonthClaimed'][i]
        day2 = carData['DayOfWeekClaimed'][i]
        year1 = carData['Year'][i]
        month1 = carData['Month'][i]
        week1 = carData['WeekOfMonth'][i]
        day1 = carData['DayOfWeek'][i]
        day_diff[i] = differ_days(get_day(year2,month2,week2,day2),get_day(year1,month1,week1,day1))

#adding column to the existing dataframe
carData['daysDiff'] = day_diff 
carData.head(3)


,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy,daysDiff
0,12,5,Wednesday,Honda,Urban,Tuesday,1,1,Female,Single,...,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability,6.0
1,1,3,Wednesday,Honda,Urban,Monday,1,4,Male,Single,...,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision,5.0
2,10,5,Friday,Honda,Urban,Thursday,11,2,Male,Married,...,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision,13.0


In [6]:
#now drop the original attibutes, like 'Month' column(we don't need anymore)
carData.drop(['Month'],axis=1,inplace=True)
carData.drop(['MonthClaimed'],axis=1,inplace=True)
carData.drop(['DayOfWeek'],axis=1,inplace=True)
carData.drop(['DayOfWeekClaimed'],axis=1,inplace=True)
carData.drop(['WeekOfMonth'],inplace=True,axis=1)
carData.drop(['WeekOfMonthClaimed'],inplace=True,axis=1)
carData.drop(['PolicyType'],axis=1,inplace=True)
carData.drop(['PolicyNumber'],inplace=True,axis=1)
carData.drop(['RepNumber'],axis=1,inplace=True)
carData.drop(['Deductible'],inplace=True,axis=1)
carData.drop(['AgeOfPolicyHolder'],inplace=True,axis=1)
carData.drop(['Year'],inplace=True,axis=1)


In [7]:
carData.head()

,Make,AccidentArea,Sex,MaritalStatus,Age,Fault,VehicleCategory,VehiclePrice,FraudFound,DriverRating,...,PastNumberOfClaims,AgeOfVehicle,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,BasePolicy,daysDiff
0,Honda,Urban,Female,Single,21,Policy Holder,Sport,"more than 69,000",No,1,...,none,3 years,No,No,External,none,1 year,3 to 4,Liability,6.0
1,Honda,Urban,Male,Single,34,Policy Holder,Sport,"more than 69,000",No,4,...,none,6 years,Yes,No,External,none,no change,1 vehicle,Collision,5.0
2,Honda,Urban,Male,Married,47,Policy Holder,Sport,"more than 69,000",No,3,...,1,7 years,No,No,External,none,no change,1 vehicle,Collision,13.0
3,Toyota,Rural,Male,Married,65,Third Party,Sport,"20,000 to 29,000",No,2,...,1,more than 7,Yes,No,External,more than 5,no change,1 vehicle,Liability,20.0
4,Honda,Urban,Female,Single,27,Third Party,Sport,"more than 69,000",No,1,...,none,5 years,No,No,External,none,no change,1 vehicle,Collision,15.0


In [8]:
#get the lable of the datasets
label_Number = LabelEncoder()  #object of lable encoder

#conver label to Number
carData['FraudFound'] = label_Number.fit_transform(carData['FraudFound'].astype('str'))

yLable= carData['FraudFound']

#drop the lable from the dataset
carData.drop(['FraudFound'],inplace=True,axis=1)


In [9]:
#change the data type to numeric
carData['Age']= pd.to_numeric(carData['Age'])
carData['DriverRating']= pd.to_numeric(carData['DriverRating'])
carData['daysDiff']= pd.to_numeric(carData['daysDiff'])


In [10]:
#select all the attributes of type object
carObject= carData.select_dtypes(include=['object']).copy()

#drop the attributes of type object
carData.drop(carData.select_dtypes(['object']),inplace=True,axis=1)

#normalization of feature to bring the value in the range [0,1]
minMaxScale= MinMaxScaler() #minMax scaler
carData= minMaxScale.fit_transform(carData)

#converting numpyarry to dataframe
carData= pd.DataFrame(carData)


/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [11]:
#one hot encoding
car_copy = carObject.copy()
car_copy = pd.get_dummies(car_copy,columns=['Make'],prefix=['Make'])
car_copy = pd.get_dummies(car_copy,columns=['AccidentArea'],prefix=['AccidentArea'])
car_copy = pd.get_dummies(car_copy,columns=['Sex'],prefix=['Sex'])
car_copy = pd.get_dummies(car_copy,columns=['MaritalStatus'],prefix=['MartalStatus'])
car_copy = pd.get_dummies(car_copy,columns=['Fault'],prefix=['Fault'])
car_copy = pd.get_dummies(car_copy,columns=['VehicleCategory'],prefix=['VehicleCategory'])
car_copy = pd.get_dummies(car_copy,columns=['VehiclePrice'],prefix=['VehiclePrice'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Accident'],prefix=['Days:Policy-Accident'])
car_copy = pd.get_dummies(car_copy,columns=['Days:Policy-Claim'],prefix=['Days:Policy-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['PastNumberOfClaims'],prefix=['PastNumberOfClaims'])
car_copy = pd.get_dummies(car_copy,columns=['AgeOfVehicle'],prefix=['AgeOfVehicle'])
car_copy = pd.get_dummies(car_copy,columns=['PoliceReportFiled'],prefix=['PoliceReportFiled'])
car_copy = pd.get_dummies(car_copy,columns=['WitnessPresent'],prefix=['WitnessPresent'])
car_copy = pd.get_dummies(car_copy,columns=['AgentType'],prefix=['AgentType'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfSuppliments'],prefix=['NumberOfSuppliments'])
car_copy = pd.get_dummies(car_copy,columns=['AddressChange-Claim'],prefix=['AddressChange-Claim'])
car_copy = pd.get_dummies(car_copy,columns=['NumberOfCars'],prefix=['NumberOfCars'])
car_copy = pd.get_dummies(car_copy,columns=['BasePolicy'],prefix=['BasePolicy'])

In [12]:
#putt all the attributes together
carObject= car_copy
carObject['Age']= carData[0]
carObject['DriverRating']= carData[1]
carObject['DaysDiff']= carData[2]
carObject['Lable']= yLable

In [13]:
#get the lable from the dataset
carLable= carObject['Lable']
carObject.drop(['Lable'],inplace=True,axis=1) #drop the lable;


In [15]:
carDateNormalized= carObject
carDateNormalized.head()

,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability,Age,DriverRating,DaysDiff
0,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0.2625,0.000000,0.071795
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0.4250,1.000000,0.069231
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0.5875,0.666667,0.089744
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.8125,0.333333,0.107692
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0.3375,0.000000,0.094872


In [16]:
#Divide the data into fraudulent and non-fraudulent data
columnlable= list(carDateNormalized.columns.values)

#create the dataframe for fraudulent and non-fraudulent data
nonFraudulent= pd.DataFrame(columns=columnlable)
nonFraudulentLable= pd.DataFrame(columns=['lable'])

fraudulent= pd.DataFrame(columns=columnlable)
fraudulentLable= pd.DataFrame(columns=['lable'])


j= 0
k= 0
for i in range(carDateNormalized.shape[0]):
    if(carLable[i]==0):
        nonFraudulent.loc[j]= carDateNormalized.loc[i]
        nonFraudulentLable.loc[j]= 0.0
        j += 1
    else:
        fraudulent.loc[k]= carDateNormalized.loc[i]
        fraudulentLable.loc[i]= 1.0
        k += 1

In [17]:
print(nonFraudulent.shape)
print(fraudulent.shape)
nonFraudulent.head()

(14496, 84)
(923, 84)


,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,NumberOfCars_2 vehicles,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability,Age,DriverRating,DaysDiff
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.2625,0.000000,0.071795
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.4250,1.000000,0.069231
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.5875,0.666667,0.089744
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.8125,0.333333,0.107692
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.3375,0.000000,0.094872


In [18]:
#convert to dataframe
nonFraudulentLable= pd.DataFrame(nonFraudulentLable)
fraudulentLable= pd.DataFrame(fraudulentLable)


In [20]:
#divide the nonFraudulent into test and train
nonFraudX_train,nonFraudX_test,nonFraudY_train,nonFraudY_test = train_test_split(nonFraudulent,nonFraudulentLable,random_state=3,test_size=0.25)
print('nonFraudxtrain:',type(nonFraudX_train))
print(nonFraudX_test.shape)

#divide the trainset into trainset and validation set
nonFraudX_trainNew,nonFraudX_Valid,nonFraudY_trainNew,nonFraudY_Valid= train_test_split(nonFraudX_train,nonFraudY_train,random_state=3,test_size=0.2)


nonFraudxtrain: <class 'pandas.core.frame.DataFrame'>
(3624, 84)


In [21]:
#convert into numpy
nonFraudX_trainNew= nonFraudX_trainNew.values
nonFraudY_trainNew= nonFraudY_trainNew.values

nonFraudX_test= nonFraudX_test.values
nonFraudY_test= nonFraudY_test.values

fraudFeatures= fraudulent.values
fraudLabel= fraudulentLable.values

nonFraudX_Valid= nonFraudX_Valid.values
nonFraudY_Valid= nonFraudY_Valid.values

In [22]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [24]:
#make the  nonfraudulent dataset for train and test loader
myNonFraudulentTrainDataset= myDataset(nonFraudX_trainNew,nonFraudY_trainNew)
myNonFraudulentTestDataset= myDataset(nonFraudX_test,nonFraudY_test)

#make validation loader
myNonFraudulentValidation= myDataset(nonFraudX_Valid,nonFraudY_Valid)

#make the fraudulent dataset for train and test loader
myFraudulentDataset= myDataset(fraudFeatures,fraudLabel)


In [25]:
#make the trainloader and test loader for nonfraudulent dataset.
nonFraudulentTrainLoader= torch.utils.data.DataLoader(myNonFraudulentTrainDataset,batch_size=8,shuffle=True,num_workers=0)
nonFraudulentTestLoader= torch.utils.data.DataLoader(myNonFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)

#make the trainloader and test loader for nonfraudulent dataset.
nonFraudulentValidationLoader= torch.utils.data.DataLoader(myNonFraudulentValidation,batch_size=8,shuffle=True,num_workers=0)

#make the trainloader and test loader for nonfraudulent dataset.
fraudulentLoader= torch.utils.data.DataLoader(myFraudulentDataset,batch_size=1,shuffle=True,num_workers=0)

In [ ]:
########Definition of the architecture of our encoder and decoder model with all the assisting functions

class Net(nn.Module):
    def __init__(self, num_latent):
        super().__init__()
        
        #So here we will first define layers for encoder network
        self.encoder = nn.Sequential(nn.Linear(84,42),
                                     nn.ReLU(),
                                     nn.Linear(42,28),
                                     nn.ReLU())
        
        #These two layers are for getting logvar and mean
        self.fc1 = nn.Linear(28, 14)
        self.act1 = nn.ReLU()
        self.mean = nn.Linear(14, num_latent)
        self.var = nn.Linear(14, num_latent)
        
        #######The decoder part
        #This is the first layer for the decoder part
        self.expand = nn.Linear(num_latent, 14)
        self.act2 = nn.ReLU()
        self.fc2 = nn.Linear(14, 28)
        self.decoder = nn.Sequential(nn.Linear(28,42),
                                     nn.ReLU(),
                                     nn.Linear(42,56),
                                     nn.Sigmoid())
        
    def enc_func(self, x):
        #here we will be returning the logvar(log variance) and mean of our network
        x = self.encoder(x)
        #x = x.view([-1, 784])
        #x = F.dropout2d(self.fc1(x), 0.5)
        
        #print("in enc_func")
        #print("x.size:",x.size())
        x = self.fc1(x)
        x = self.act1(x)
        
        mean = self.mean(x)
        logvar = self.var(x)
        return mean, logvar
    
    def dec_func(self, z):
        #here z is the latent variable state
        z = self.expand(z)
        #z = F.dropout2d(self.fc3(z), 0.5)
        
        z= self.act2(z)
        z = self.fc2(z)
        #z = z.view([-1, 16, 7, 7])
        
        out = self.decoder(z)
        out = F.sigmoid(out)
        return out
    
    def get_hidden(self, mean, logvar):
        
        if self.training:
            #print("inside get_hidden")
            std = torch.exp(0.5*logvar)   #So as to get std
            noise = torch.randn_like(mean)   #So as to get the noise of standard distribution
            return noise.mul(std).add_(mean)
        else:
            return mean
    
    def forward(self, x):
        mean, logvar = self.enc_func(x)
        #print("mean:",mean)
        z = self.get_hidden(mean, logvar)
        
        #print("z.size():",z.size())
        out = self.dec_func(z)
        return out, mean, logvar

In [ ]:
#######This is the custom loss function defined for VAE
### You can even refere to: https://github.com/pytorch/examples/pull/226 

def VAE_loss(out, target, mean, logvar):
    category1 = nn.BCELoss()
    bce_loss = category1(out, target)
    #print("out.size():",out.size())
    
    #We will scale the following losses with this factor
    scaling_factor = out.shape[0]*out.shape[1]
    
    ####Now we are gonna define the KL divergence loss
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    kl_loss = -0.5 * torch.sum(1 + logvar - mean**2 - torch.exp(logvar))
    kl_loss /= scaling_factor
    
    return bce_loss + kl_loss

In [ ]:
#####The function which we will call for training our model

def train(trainloader, iters, model,optimizer):
    
    for i in range(iters):
        model.train()
        runningLoss= 0
        #model.to(device)
        for feature,label in trainloader:
            #images = images.to(device)
            inputs= torch.tensor(feature)
            #print("inputs:",inputs)
            #print("inputs.size():",inputs.size())
            
            optimizer.zero_grad()
            out, mean, logvar = model(feature)
            
            loss = VAE_loss(out, inputs, mean, logvar)
            runningLoss += loss.item()
            loss.backward()
            optimizer.step()
        print('Epoch--[%d] --------------------Training loss: %.6f' %(i + 1,  runningLoss /nonFraudX_trainNew.shape[0]))
    
        
            
        

In [ ]:
######Setting all the hyperparameters
##You can change them if you want

iters = 50
num_latent = 5
#print_every = 5    #print after every 5 iterations
model = Net(num_latent)

#device = ('cuda' if torch.cuda.is_available() else 'cpu')
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=1e-2,momentum=0.9,weight_decay=5e-4)

train(nonFraudulentTrainLoader, iters, model,optimizer)

In [ ]:
def testmodel(loader):
    for feature,lable in loader:
        inputs= torch.tensor(feature)
        out, mean, logvar = model(feature)
        loss = VAE_loss(out, inputs, mean, logvar)
        print("Loss:",loss)
        
        

In [ ]:
testmodel(fraudulentTestLoader)